## Focusing on terse `FeatNets` training/eval
9.28.2020

In [1]:
import os, sys
import copy as copyroot
import pandas as pd
from IPython.display import display
from matplotlib import pyplot as plt
from fastai2.basics import *
from fastai2.vision.all import *
%load_ext autoreload
%autoreload 2

from module.mnist_helpers import build_df, eda_fig_1, build_dls
from module.mnist_models import FeatsNet
from module.mnist_metrics import metrics_df

In [2]:
dls_tl = build_dls(target='topleft')
dls_cr = build_dls(target='center')

### Grid Fit

In [9]:
EPOCHS = 20
TMP_FN = 'assets/tmp.csv'

feats = [
         ('pix', 'pts11'),
         ('pix', 'pts12'),
         ('pix', 'pts22'),
         ('', 'pts11'),
         ('', 'pts12'),
         ('', 'pts22'),
        ]

dls = [dls_tl, dls_cr]

list_df_metrics = []

t0 = time.time()

for dl_ind in (0,1):
    for _feat in feats:

        featsnet  = FeatsNet(feats=_feat)
        tmplogger = CSVLogger(fname=TMP_FN)
        learn     = Learner(dls_tl, 
                            featsnet, 
                            loss_func=MSELossFlat(), 
                            cbs=[tmplogger])

        with learn.no_logging(): learn.fit(EPOCHS)

        _df = metrics_df(
                   learn, 
                   s_model =    f'FeatsNet1.{_feat[-1][-2:]}',
                   s_details =  f'feats={",".join(_feat)}',
                   s_target =   "topleft" if dl_ind == 0 else "center",
                  )
        
        list_df_metrics.append(_df)
        
print(f'done in {round(time.time() - t0,0)} secs ')
print(f'{len(list_df_metrics)} metric_dfs built')

done in 561.0 secs 
12 metric_dfs built


In [12]:
df = pd.concat(list_df_metrics)
df.reset_index(inplace=True, drop=True)
df.sort_values(by='mse', ascending=True)

,model,details,target,split,mse,mae,r2,dist_avg,dist_r2,sqdist_avg,sqdist_r2
5,FeatsNet1.22,"feats=pix,pts22",topleft,train,0.006158,0.057115,0.787057,0.092237,0.607826,0.012316,0.837864
17,FeatsNet1.22,"feats=pix,pts22",center,train,0.007534,0.062097,0.739600,0.100464,0.572842,0.015067,0.801649
15,FeatsNet1.12,"feats=pix,pts12",center,train,0.007849,0.063626,0.754144,0.103412,0.560312,0.015699,0.793337
3,FeatsNet1.12,"feats=pix,pts12",topleft,train,0.008364,0.065627,0.713868,0.106106,0.548855,0.016727,0.779795
13,FeatsNet1.11,"feats=pix,pts11",center,train,0.009264,0.070066,0.644155,0.112525,0.521563,0.018527,0.756101
1,FeatsNet1.11,"feats=pix,pts11",topleft,train,0.009834,0.072080,0.606355,0.115771,0.507763,0.019668,0.741083
4,FeatsNet1.22,"feats=pix,pts22",topleft,valid,0.013204,0.076632,0.566738,0.126752,0.413290,0.026409,0.586774
16,FeatsNet1.22,"feats=pix,pts22",center,valid,0.014107,0.078245,0.543622,0.129522,0.400468,0.028214,0.558536
2,FeatsNet1.12,"feats=pix,pts12",topleft,valid,0.015845,0.084004,0.498059,0.139944,0.352225,0.031689,0.504155
14,FeatsNet1.12,"feats=pix,pts12",center,valid,0.015999,0.084072,0.535459,0.140841,0.348076,0.031999,0.499309


In [13]:
df.to_csv('assets/metrics-dfs/rcn6b-featsnet1x.csv')